In [104]:
import math
from random import randint, choice
from gym.envs.classic_control import CartPoleEnv
import tensorflow as tf
import pandas as pd
import numpy as np
import heapq
from random import random

In [109]:
T = 100
mu = 0.7
batch_size = 10
GAMMA = 0.5
learning_rate = 0.001

In [92]:
class Transaction:
    columns = ['state', 'action', 'reward', 'next_state', 'done', 'qvalue']
    actions = [[0, 1], [1, 0]]
    def __init__(self, item):
        self.item = dict(zip(Transaction.columns, item))
        
    def  __lt__(self, other):
        return self.item['id'] < other.item['id']
    
    def __repr__(self):
        return str(self.item)
    
    def value(self):
        return self.item['state'] + Transaction.actions[self.item['action']]
    
    def __getitem__(self, key):
        return self.item[key]
    
    def __setitem__(self, key, value):
        self.item[key] = value

class Memory:
    def __init__(self, max_size):
        MAX_TRANSACTION = Transaction(Transaction.columns)
        MAX_TRANSACTION.item['id'] = -1
        self.d = []
        self.id = 0
        self.max_size = max_size
        
    def insert(self, item):
        self.id += 1
        item['id'] = self.id
        #print(self.d)

        if len(self.d) == self.max_size:
            heapq.heapreplace(self.d, item)
        else:
            heapq.heappush(self.d, item)

    def batch(self, n):
        if len(self.d) < n:
            return np.array(heapq.nsmallest(n, self.d))
        else:
            return np.array(self.d)
        
def transform_to_Q_input(batch):
    return np.array([transaction.value() for transaction in batch])

def get_QValues(batch):
    return np.array([transaction['qvalue'] for transaction in batch])
    
D = Memory(2)
item1 = Transaction([np.array([0,1,2,3]), 1,0,np.array([3,2,1,0]),0,10])
item2 = Transaction([np.array([0,1,2,3]), 1,0,np.array([3,2,1,0]),0,10])
item3 = Transaction([np.array([0,1,2,3]), 1,0,np.array([3,2,1,0]),0,10])

D.insert(item1)
D.insert(item2)
D.insert(item3)

'''
print(D.d, D.batch(2), D.d)
item = Transaction([np.array([-1,1,2,3]), 1,0,np.array([3,2,1,0]),0,10])
D.insert(item)
print(D.d, D.batch(1))
'''

'\nprint(D.d, D.batch(2), D.d)\nitem = Transaction([np.array([-1,1,2,3]), 1,0,np.array([3,2,1,0]),0,10])\nD.insert(item)\nprint(D.d, D.batch(1))\n'

In [113]:
class QNetwork:
    def __init__(self, sess):
        self.state_size = 4
        self.action_size = 2
        self.input_size = self.state_size + self.action_size
        self.units = 4
        self.output_size = 1
        
        self.input = tf.placeholder(shape=(None, self.input_size), dtype=tf.float32)
        self.label = tf.placeholder(shape=(None), dtype=tf.float32)
        
        self.build_network()
        self.sess = sess
        self.sess.run(tf.global_variables_initializer())

    def build_network(self):
        self.layer_1 = tf.layers.dense(inputs=self.input, units=self.units)
        self.layer_2 = tf.layers.dense(inputs=self.layer_1, units=self.units/2)
        self.qvalue = tf.layers.dense(inputs=self.layer_2, units=self.output_size, activation=tf.nn.tanh)

        self.loss = tf.losses.mean_squared_error(labels=self.label, predictions=self.qvalue)
        self.train_step = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
        self.best_action = tf.argmax(self.qvalue, axis=1)
        
    def get_Q_value(self, batch):
        return self.sess.run(self.qvalue, feed_dict={self.input: batch})
        
    def get_best_action(self, state):
        batch = [0, 0]
        batch[0] = state + [0, 1]
        batch[1] = state + [1, 0]
        #print state, np.array(batch).shape
        
        best_action = self.sess.run(self.best_action, feed_dict={self.input: batch})
        return best_action
    
    def update(self, batch):
        inputs = transform_to_Q_input(batch)
        labels = get_QValues(batch)
        print(self.sess.run([self.train_step, self.loss], feed_dict={self.input: inputs, self.label: labels}))
        

In [114]:
def add_qvalue(batch, Q, gamma):
    _batch = transform_to_Q_input(batch)
    qvalues = Q.get_Q_value(_batch)
    for qvalue,transaction in zip(qvalues, batch):
        if not transaction['done']:
            transaction['qvalue'] = transaction['reward'] + qvalue*gamma
        else:
            transaction['qvalue'] = transaction['reward']

In [116]:
D = Memory(T)
sess = tf.Session()
Q = QNetwork(sess)

env = CartPoleEnv()
#env.theta_threshold_radians = 2 * math.pi * 1/4
state = env.reset()
state = state.tolist()

for _ in range(10000):
    #env.render()
    #action = env.action_space.sample()
    if random() > mu:
        action = env.action_space.sample()
    else:
        actoin = Q.get_best_action(state)
    
    next_state, reward, done, _ = env.step(action) # take a random action
    next_state = next_state.tolist()
    
    transaction = Transaction((state, action, reward, next_state, done, -1))
    D.insert(transaction)
    state = next_state
    
    batch = D.batch(batch_size)
    add_qvalue(batch, Q, GAMMA)
    Q.update(batch)
    
    #print(transaction, batch)
    
    if done:
        state = env.reset()
        state = state.tolist()

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
[None, 1.0365754]
[None, 1.017228]
[None, 0.9987379]
[None, 0.98094845]
[None, 0.9637286]
[None, 0.93469125]
[None, 0.92709386]
[None, 0.916861]
[None, 0.89207685]
[None, 0.8961821]
[None, 0.8957388]
[None, 0.89223075]
[None, 0.8865821]
[None, 0.8793884]
[None, 0.87104607]
[None, 0.8589957]
[None, 0.8486682]
[None, 0.8277637]
[None, 0.8256235]
[None, 0.8248285]
[None, 0.8252383]
[None, 0.8267529]
[None, 0.8293082]
[None, 0.83286834]
[None, 0.8374181]
[None, 0.84295285]
[None, 0.8494675]
[None, 0.8600713]
[None, 0.8557909]
[None, 0.852536]
[None, 0.85025287]
[None, 0.8488841]
[None, 0.8514272]
[None, 0.8518304]
[None, 0.85055256]
[None, 0.84584236]
[None, 0.8421131]
[None, 0.83933663]
[None, 0.8374713]
[None, 0.8364682]
[None, 0.83617246]
[None, 0.83019763]
[None, 0.8266646]
[None, 0.8222031]
[None, 0.8170437]
[None, 0.8113679]
[None, 0.80531716]
[None, 0.7990012]
[None, 0.7925053]
[None, 0

[None, 0.25827008]
[None, 0.25825423]
[None, 0.2582552]
[None, 0.25823602]
[None, 0.25820675]
[None, 0.2581664]
[None, 0.2580522]
[None, 0.2579551]
[None, 0.2578542]
[None, 0.25505847]
[None, 0.25754642]
[None, 0.2574184]
[None, 0.257292]
[None, 0.2571672]
[None, 0.25704378]
[None, 0.25692174]
[None, 0.25680104]
[None, 0.25668174]
[None, 0.2565307]
[None, 0.2536185]
[None, 0.25347036]
[None, 0.25594422]
[None, 0.25583243]
[None, 0.25572208]
[None, 0.25561294]
[None, 0.25547263]
[None, 0.25531733]
[None, 0.25242457]
[None, 0.25227284]
[None, 0.2521506]
[None, 0.25203505]
[None, 0.25192496]
[None, 0.25181964]
[None, 0.2517519]
[None, 0.25166982]
[None, 0.25158897]
[None, 0.2515092]
[None, 0.25147235]
[None, 0.2514506]
[None, 0.25135225]
[None, 0.25123215]
[None, 0.251096]
[None, 0.2509874]
[None, 0.25088456]
[None, 0.25078732]
[None, 0.2481218]
[None, 0.24806279]
[None, 0.24802117]
[None, 0.24796002]
[None, 0.24789903]
[None, 0.24783857]
[None, 0.2477396]
[None, 0.24763262]
[None, 0.2475

[None, 0.23889199]
[None, 0.24144536]
[None, 0.24143979]
[None, 0.24141997]
[None, 0.24140346]
[None, 0.2413914]
[None, 0.24138442]
[None, 0.23885298]
[None, 0.23885562]
[None, 0.23885982]
[None, 0.24141493]
[None, 0.24139874]
[None, 0.24138147]
[None, 0.24136357]
[None, 0.24134506]
[None, 0.24132647]
[None, 0.24130785]
[None, 0.23873803]
[None, 0.23873672]
[None, 0.23873748]
[None, 0.24130888]
[None, 0.24130464]
[None, 0.24129821]
[None, 0.24127847]
[None, 0.24124038]
[None, 0.24120156]
[None, 0.24116333]
[None, 0.24112639]
[None, 0.24109189]
[None, 0.24107173]
[None, 0.23853184]
[None, 0.23853752]
[None, 0.23854233]
[None, 0.2385455]
[None, 0.23854604]
[None, 0.23852955]
[None, 0.23851052]
[None, 0.23848955]
[None, 0.23845832]
[None, 0.23842803]
[None, 0.2383989]
[None, 0.23581962]
[None, 0.23580565]
[None, 0.23579465]
[None, 0.23579572]
[None, 0.23581316]
[None, 0.23582329]
[None, 0.23583461]
[None, 0.23584662]
[None, 0.23585874]
[None, 0.23842178]
[None, 0.23841235]
[None, 0.238401

[None, 0.23647597]
[None, 0.23650134]
[None, 0.23652625]
[None, 0.23654996]
[None, 0.23910025]
[None, 0.23910971]
[None, 0.23911738]
[None, 0.23912334]
[None, 0.23912768]
[None, 0.2391307]
[None, 0.23660417]
[None, 0.23660398]
[None, 0.23660561]
[None, 0.23913643]
[None, 0.23912947]
[None, 0.23912515]
[None, 0.23912416]
[None, 0.23912698]
[None, 0.23913406]
[None, 0.23664063]
[None, 0.23663893]
[None, 0.23664515]
[None, 0.23665157]
[None, 0.23919058]
[None, 0.23918669]
[None, 0.2391769]
[None, 0.23916858]
[None, 0.2391669]
[None, 0.23917203]
[None, 0.23916967]
[None, 0.23916137]
[None, 0.23916224]
[None, 0.2391686]
[None, 0.23917334]
[None, 0.23916994]
[None, 0.23916583]
[None, 0.23916137]
[None, 0.23915708]
[None, 0.23915254]
[None, 0.23914751]
[None, 0.23914228]
[None, 0.23912978]
[None, 0.23911907]
[None, 0.23911028]
[None, 0.23910998]
[None, 0.23911092]
[None, 0.23911288]
[None, 0.24161863]
[None, 0.2391196]
[None, 0.23911268]
[None, 0.23910752]
[None, 0.23910405]
[None, 0.23910274

[None, 0.23575507]
[None, 0.2357565]
[None, 0.23575513]
[None, 0.23575391]
[None, 0.23575285]
[None, 0.23574868]
[None, 0.23574528]
[None, 0.23574293]
[None, 0.23574187]
[None, 0.23574233]
[None, 0.23575318]
[None, 0.23576485]
[None, 0.2357704]
[None, 0.23577681]
[None, 0.23329675]
[None, 0.23329927]
[None, 0.23329812]
[None, 0.23329678]
[None, 0.23329522]
[None, 0.23328975]
[None, 0.23328458]
[None, 0.23328003]
[None, 0.2332763]
[None, 0.2332737]
[None, 0.23076998]
[None, 0.23076747]
[None, 0.23076504]
[None, 0.23076536]
[None, 0.23076245]
[None, 0.23075938]
[None, 0.23326267]
[None, 0.23326269]
[None, 0.23326008]
[None, 0.2332584]
[None, 0.23325789]
[None, 0.2307643]
[None, 0.23076464]
[None, 0.2307644]
[None, 0.23076715]
[None, 0.23076685]
[None, 0.2307643]
[None, 0.23076215]
[None, 0.23076065]
[None, 0.23075984]
[None, 0.23327334]
[None, 0.23327175]
[None, 0.23327066]
[None, 0.23327047]
[None, 0.23327114]
[None, 0.23076704]
[None, 0.23076552]
[None, 0.23076455]
[None, 0.23076712]
[

...

[None, 0.24000762]
[None, 0.24000764]
[None, 0.24000767]
[None, 0.24000767]
[None, 0.24000774]
[None, 0.24000771]
[None, 0.24000767]
[None, 0.2400076]
[None, 0.24000742]
[None, 0.2400074]
[None, 0.24000737]
[None, 0.23750761]
[None, 0.24000771]
[None, 0.24000776]
[None, 0.24000776]
[None, 0.24000786]
[None, 0.24000798]
[None, 0.24000816]
[None, 0.24000816]
[None, 0.24000818]
[None, 0.24000818]
[None, 0.24000816]
[None, 0.24000816]
[None, 0.24000816]
[None, 0.24000818]
[None, 0.24000818]
[None, 0.23750801]
[None, 0.23750795]
[None, 0.23750801]
[None, 0.23750801]
[None, 0.23750798]
[None, 0.23750795]
[None, 0.23750791]
[None, 0.23750791]
[None, 0.23750798]
[None, 0.23750795]
[None, 0.23500791]
[None, 0.23500781]
[None, 0.23500776]
[None, 0.2350078]
[None, 0.23500781]
[None, 0.23500781]
[None, 0.23500773]
[None, 0.23500757]
[None, 0.23500766]
[None, 0.23250772]
[None, 0.23250772]
[None, 0.23250777]
[None, 0.23250774]
[None, 0.23250766]
[None, 0.23250766]
[None, 0.23250769]
[None, 0.232507

[None, 0.23500493]
[None, 0.23500498]
[None, 0.23500496]
[None, 0.23500498]
[None, 0.23500498]
[None, 0.2350049]
[None, 0.23250483]
[None, 0.2325049]
[None, 0.23250495]
[None, 0.23250493]
[None, 0.23250498]
[None, 0.23250493]
[None, 0.23250493]
[None, 0.23250495]
[None, 0.23250498]
[None, 0.23000483]
[None, 0.23000494]
[None, 0.23000498]
[None, 0.2300051]
[None, 0.23000506]
[None, 0.23000503]
[None, 0.23000488]
[None, 0.23000488]
[None, 0.23000494]
[None, 0.23000498]
[None, 0.23000488]
[None, 0.23000483]
[None, 0.23000479]
[None, 0.23000488]
[None, 0.2300048]
[None, 0.23000479]
[None, 0.23000483]
[None, 0.23000494]
[None, 0.23000498]
[None, 0.23000509]
[None, 0.23000509]
[None, 0.23000517]
[None, 0.23000517]
[None, 0.23000513]
[None, 0.23250502]
[None, 0.23250501]
[None, 0.23000488]
[None, 0.23000494]
[None, 0.23000498]
[None, 0.2300049]
[None, 0.23000494]
[None, 0.23000495]
[None, 0.2300049]
[None, 0.23000488]
[None, 0.23000495]
[None, 0.23000498]
[None, 0.23000498]
[None, 0.22750498]

[None, 0.23250356]
[None, 0.23250352]
[None, 0.23250349]
[None, 0.23000337]
[None, 0.2300033]
[None, 0.23000327]
[None, 0.23000322]
[None, 0.23000322]
[None, 0.23000318]
[None, 0.23250327]
[None, 0.23250322]
[None, 0.2325032]
[None, 0.23000322]
[None, 0.23000327]
[None, 0.23000339]
[None, 0.23000333]
[None, 0.23000333]
[None, 0.23000333]
[None, 0.23250318]
[None, 0.23250313]
[None, 0.2325031]
[None, 0.23250313]
[None, 0.23000307]
[None, 0.23000312]
[None, 0.23000312]
[None, 0.2300031]
[None, 0.23000303]
[None, 0.23000306]
[None, 0.230003]
[None, 0.23000303]
[None, 0.23000306]
[None, 0.23000307]
[None, 0.23000318]
[None, 0.23000312]
[None, 0.23000318]
[None, 0.2300032]
[None, 0.2300032]
[None, 0.23000325]
[None, 0.2300033]
[None, 0.23000333]
[None, 0.23000322]
[None, 0.23000327]
[None, 0.23000322]
[None, 0.23250322]
[None, 0.23250322]
[None, 0.23250322]
[None, 0.23250318]
[None, 0.2325031]
[None, 0.23250307]
[None, 0.23250306]
[None, 0.23250303]
[None, 0.232503]
[None, 0.23500301]
[None

[None, 0.23500164]
[None, 0.23500165]
[None, 0.23500164]
[None, 0.23500165]
[None, 0.23500171]
[None, 0.23500171]
[None, 0.23500176]
[None, 0.23500171]
[None, 0.2350018]
[None, 0.2350018]
[None, 0.23500171]
[None, 0.23500168]
[None, 0.23500165]
[None, 0.23500164]
[None, 0.23500161]
[None, 0.23750167]
[None, 0.23750174]
[None, 0.23750186]
[None, 0.235002]
[None, 0.23500206]
[None, 0.23500206]
[None, 0.23500206]
[None, 0.235002]
[None, 0.235002]
[None, 0.235002]
[None, 0.23750198]
[None, 0.23750193]
[None, 0.23500192]
[None, 0.23500198]
[None, 0.2350019]
[None, 0.2350019]
[None, 0.23500186]
[None, 0.23500183]
[None, 0.23500186]
[None, 0.23500186]
[None, 0.23500186]
[None, 0.23500188]
[None, 0.23500183]
[None, 0.23500186]
[None, 0.2350019]
[None, 0.23500188]
[None, 0.23750186]
[None, 0.2350019]
[None, 0.2350019]
[None, 0.23500195]
[None, 0.23500198]
[None, 0.235002]
[None, 0.235002]
[None, 0.23500203]
[None, 0.2350019]
[None, 0.2350019]
[None, 0.23250206]
[None, 0.23250206]
[None, 0.23250

[None, 0.23250085]
[None, 0.23250085]
[None, 0.23250084]
[None, 0.23250088]
[None, 0.23250084]
[None, 0.23250084]
[None, 0.23250085]
[None, 0.23250085]
[None, 0.23250088]
[None, 0.23250091]
[None, 0.23250084]
[None, 0.23250085]
[None, 0.23250088]
[None, 0.23250088]
[None, 0.23000085]
[None, 0.23000078]
[None, 0.23000076]
[None, 0.23000073]
[None, 0.23000073]
[None, 0.23000078]
[None, 0.23000078]
[None, 0.23000082]
[None, 0.23250084]
[None, 0.23000088]
[None, 0.23000088]
[None, 0.2300009]
[None, 0.23000097]
[None, 0.23000097]
[None, 0.230001]
[None, 0.23000103]
[None, 0.230001]
[None, 0.22750098]
[None, 0.227501]
[None, 0.23000097]
[None, 0.23000097]
[None, 0.23000103]
[None, 0.23000097]
[None, 0.23000103]
[None, 0.23000105]
[None, 0.23000109]
[None, 0.22750108]
[None, 0.22750105]
[None, 0.22750108]
[None, 0.23000108]
[None, 0.23000108]
[None, 0.23000109]
[None, 0.23000108]
[None, 0.23000109]
[None, 0.23000109]
[None, 0.23000112]
[None, 0.23000108]
[None, 0.23000108]
[None, 0.23000105]


In [91]:
env.reset()
for _ in range(1000):
    #env.render()
    actoin = Q.get_best_action(state)
    
    next_state, reward, done, _ = env.step(action) # take a random action
    next_state = next_state.tolist()
    print(next_state, reward)
    
    if done:
        break

([0.03325931826711125, -0.1800216926925826, -0.012120367192087696, 0.2842336338884332], 1.0)
([0.0296588844132596, -0.37496869702879376, -0.006435694514319032, 0.5730693611773274], 1.0)
([0.022159510472683726, -0.569999824439548, 0.005025692709227517, 0.8637179128149124], 1.0)
([0.010759513983892766, -0.7651898317521835, 0.022300050965525765, 1.1579767737067883], 1.0)
([-0.004544282651150904, -0.9605952126135293, 0.04545958643966153, 1.4575676394343404], 1.0)
([-0.023756186903421492, -1.1562444676935333, 0.07461093922834834, 1.7640988470804295], 1.0)
([-0.046881076257292154, -1.352126523103784, 0.10989291616995693, 2.0790197534438115], 1.0)
([-0.07392360671936783, -1.548176824081446, 0.15147331123883317, 2.403564824903905], 1.0)
([-0.10488714320099675, -1.7442608882016808, 0.19954460773691127, 2.738686181812374], 1.0)
([-0.13977236096503037, -1.9401555403675912, 0.25431833137315873, 3.0849749992790327], 1.0)


In [29]:
action = env.action_space.sample()
print(action)
env.step(action)

0


(array([ 0.04173736, -0.17609484, -0.02005421,  0.30048321]), 1.0, False, {})